In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import folium
from folium.plugins import HeatMap
from collections import Counter
from matplotlib import font_manager, rc
from branca.colormap import linear
from branca.colormap import LinearColormap
import numpy as np
from collections import defaultdict

In [2]:
# 데이터 불러오기
gwangjin = pd.read_csv('./data/make_file/광진구_모든_시설의_위치.csv', index_col=0)

In [6]:
# 위도와 경도별 빈도 계산
heat_data = gwangjin[['위도', '경도']].values.tolist()

# 좌표별 빈도를 계산 (같은 위치가 여러 번 등장하면 빈도가 높아짐)
heat_counts = Counter(map(tuple, heat_data))

# 빈도의 최대값을 90백분위수로 설정 (이상치를 피하기 위함)
freq_values = list(heat_counts.values())
max_freq = np.percentile(freq_values, 90)  # 90th percentile

# 색상 범위 설정 (LinearColormap 사용)
color_map = LinearColormap(
    colors=['blue', 'green', 'yellow', 'orange', 'red'],
    vmin=0, vmax=max_freq
)

# 지도 중심 좌표 설정 (광진구 내 평균 좌표)
map_center = [gwangjin['위도'].mean(), gwangjin['경도'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 히트맵 데이터의 빈도를 0~1로 정규화하여 색상 적용
normalized_heat_data = []
for lat, lon in heat_data:
    # 좌표의 빈도 구하기
    freq = heat_counts[(lat, lon)]
    # 빈도를 0~1로 정규화
    normalized_freq = (freq - 0) / (max_freq - 0)
    normalized_heat_data.append([lat, lon, normalized_freq])

# 히트맵 추가
HeatMap(normalized_heat_data, radius=10, gradient=color_map).add_to(m)

# 색상 범례 추가
color_map.add_to(m)

# 지도 출력
m